In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
sns.set_theme(style="ticks", font="Linux Biolinum O", font_scale=1.2)

frames = [pd.read_csv(data) for data in glob('../data/operation-*.txt')]
data = pd.concat(frames, ignore_index=True)
data = data[data['comment'].isna()]
data = data[(data['inner_k'] == 32) & (data['inner_n'] == 80) & (data['outer_k'] == 8) & (data['outer_n'] == 10)]

def protocol_operation(row):
    return f'{row["protocol"]} - {row["operation"]}'

data["protocol - operation"] = data.apply(protocol_operation, axis=1)
data

In [ ]:
hue_order = ["entropy - put", "kademlia - put", "entropy - get", "kademlia - get"]
entropy_palette = sns.cubehelix_palette(n_colors=4)
kademlia_palette = sns.cubehelix_palette(start=.8, rot=-.5, n_colors=4)
palette = [c for p in zip(entropy_palette, kademlia_palette) for c in p][:4]

sns.relplot(data=data[data['operation'] != 'repair'],
    kind="line",
    x="num_concurrent", y="latency", hue="protocol - operation",
    hue_order=hue_order, palette=palette,
    height=2.56, aspect=2.) \
.set_xlabels("Concurrent Operation") \
.set_ylabels("Latency (s)") \
.savefig("scalable-put-get.pdf")

In [ ]:
hue_order = ["entropy - repair", "kademlia - repair"]
entropy_palette = sns.cubehelix_palette(n_colors=4)
kademlia_palette = sns.cubehelix_palette(start=.8, rot=-.5, n_colors=4)
palette = [c for p in zip(entropy_palette, kademlia_palette) for c in p][4:6]

sns.relplot(data=data[data['operation'] == 'repair'],
    kind="line",
    x="num_concurrent", y="latency", hue="protocol - operation",
    hue_order=hue_order, palette=palette,
    height=2.56, aspect=2.) \
.set_xlabels("Concurrent Operation") \
.set_ylabels("Latency (s)") \
.savefig("scalable-repair.pdf")